In [52]:
from optimizer_supervised import Optimizer
from model import TransformerModel
from Hamiltonian import Ising

In [53]:
import numpy as np
import torch

In [54]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
    print("GPU is available")
else:
    # Set the device to CPU
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

# Example usage: Move a tensor to the selected device
x = torch.tensor([1, 2, 3])
x = x.to(device)

GPU is available


In [55]:
torch.set_default_tensor_type(
    torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
)

In [56]:
system_sizes = np.arange(8, 21, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 10000

print("Sizes:", system_sizes)
print("Hamiltonians:", Hamiltonians)
print("Param dim:", param_dim)

Sizes: [[ 8]
 [10]
 [12]
 [14]
 [16]
 [18]
 [20]]
Hamiltonians: [<Hamiltonian.Ising object at 0x7cf4f035d400>, <Hamiltonian.Ising object at 0x7cf50f869460>, <Hamiltonian.Ising object at 0x7cf50fa2fc80>, <Hamiltonian.Ising object at 0x7cf4f035eba0>, <Hamiltonian.Ising object at 0x7cf50f86b7a0>, <Hamiltonian.Ising object at 0x7cf4f035fe90>, <Hamiltonian.Ising object at 0x7cf4f035d2e0>]
Param dim: 1


In [57]:
model = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

In [58]:
param_range = None
point_of_interest = None
use_SR = False

In [59]:
optim = Optimizer(model, Hamiltonians, point_of_interest=point_of_interest)

In [60]:
import os

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [61]:
# optim.train(
#     100,
#     batch=1000000,
#     max_unique=100,
#     param_range=param_range,
#     fine_tuning=False,
#     use_SR=use_SR,
#     ensemble_id=int(use_SR),
# )

In [62]:
system_size = torch.tensor([8])
param = torch.tensor([1])
model.set_param(system_size=system_size, param=param)

print(model.param)
print(model.system_size)

tensor([1])
tensor([8])


In [63]:
# H = Hamiltonians[0]
# batch = 100
# max_unique = 10
#
# loss, log_amp, log_phase, sample_weight, Er, Ei, E_var = optim.minimize_energy_step(
#     H, batch, max_unique, use_symmetry=True
# )
#
# print("Loss:", loss)

Loss: tensor(-0.0433, grad_fn=<MulBackward0>)


In [64]:
from model_utils import compute_psi

In [65]:
basis = torch.tensor(H.generate_basis())
symmetry = H.symmetry
log_amp, log_phase = compute_psi(model, basis, symmetry, check_duplicate=True)
print("Log_amp:", log_amp)
print("Log_phase:", log_phase)

Log_amp: tensor([-2.1147, -3.5025, -4.4339, -3.9888, -4.5769, -5.6326, -5.1818, -4.2123,
        -4.6144, -5.8951, -6.6344, -5.9671, -5.4398, -6.2230, -5.4674, -4.2806,
        -4.6144, -5.9483, -6.8276, -6.2891, -6.8039, -7.7233, -7.1573, -6.0576,
        -5.4992, -6.6536, -7.2473, -6.3880, -5.7522, -6.3660, -5.5076, -4.2123,
        -4.5769, -5.9275, -6.8408, -6.3435, -6.9366, -7.9246, -7.4320, -6.3880,
        -6.8039, -8.0068, -8.6798, -7.9045, -7.3736, -8.0424, -7.2328, -5.9671,
        -5.4398, -6.7046, -7.5323, -6.8710, -7.3736, -8.1560, -7.4978, -6.2891,
        -5.7522, -6.8057, -7.3242, -6.3435, -5.7287, -6.2404, -5.3412, -3.9888,
        -4.4339, -5.7963, -6.7188, -6.2404, -6.8408, -7.8543, -7.3776, -6.3660,
        -6.8276, -8.0660, -8.7722, -8.0424, -7.5323, -8.2475, -7.4618, -6.2230,
        -6.6344, -7.9249, -8.7722, -8.1560, -8.6798, -9.5130, -8.8940, -7.7233,
        -7.2473, -8.3323, -8.8733, -7.9246, -7.3242, -7.8543, -6.9746, -5.6326,
        -5.1818, -6.4998, -7.37

In [66]:
amp = torch.exp(log_amp)
phase = torch.exp(log_phase)

print("Amplitude:", amp)
print("Phase:", phase)

Amplitude: tensor([1.2067e-01, 3.0122e-02, 1.1868e-02, 1.8522e-02, 1.0286e-02, 3.5791e-03,
        5.6181e-03, 1.4812e-02, 9.9086e-03, 2.7530e-03, 1.3143e-03, 2.5616e-03,
        4.3402e-03, 1.9833e-03, 4.2224e-03, 1.3835e-02, 9.9086e-03, 2.6102e-03,
        1.0835e-03, 1.8565e-03, 1.1094e-03, 4.4238e-04, 7.7913e-04, 2.3401e-03,
        4.0899e-03, 1.2894e-03, 7.1212e-04, 1.6817e-03, 3.1757e-03, 1.7190e-03,
        4.0559e-03, 1.4812e-02, 1.0286e-02, 2.6651e-03, 1.0692e-03, 1.7582e-03,
        9.7161e-04, 3.6172e-04, 5.9201e-04, 1.6817e-03, 1.1094e-03, 3.3320e-04,
        1.6998e-04, 3.6909e-04, 6.2759e-04, 3.2153e-04, 7.2251e-04, 2.5616e-03,
        4.3402e-03, 1.2253e-03, 5.3553e-04, 1.0375e-03, 6.2759e-04, 2.8702e-04,
        5.5432e-04, 1.8565e-03, 3.1757e-03, 1.1075e-03, 6.5939e-04, 1.7582e-03,
        3.2514e-03, 1.9490e-03, 4.7902e-03, 1.8522e-02, 1.1868e-02, 3.0388e-03,
        1.2080e-03, 1.9490e-03, 1.0692e-03, 3.8807e-04, 6.2511e-04, 1.7190e-03,
        1.0835e-03, 3.1405e-0

In [67]:
psi_predicted = torch.exp(log_amp) * torch.exp(1j * log_phase)
print(psi_predicted)

tensor([1.1880e-01-2.1194e-02j, 2.9666e-02-5.2249e-03j, 1.1689e-02-2.0531e-03j,
        1.8243e-02-3.2033e-03j, 1.0131e-02-1.7808e-03j, 3.5269e-03-6.0937e-04j,
        5.5343e-03-9.6709e-04j, 1.4591e-02-2.5479e-03j, 9.7589e-03-1.7160e-03j,
        2.7122e-03-4.7215e-04j, 1.2949e-03-2.2536e-04j, 2.5236e-03-4.3972e-04j,
        4.2755e-03-7.4673e-04j, 1.9544e-03-3.3704e-04j, 4.1603e-03-7.2152e-04j,
        1.3630e-02-2.3707e-03j, 9.7589e-03-1.7160e-03j, 2.5721e-03-4.4465e-04j,
        1.0673e-03-1.8645e-04j, 1.8286e-03-3.2076e-04j, 1.0926e-03-1.9202e-04j,
        4.3595e-04-7.5161e-05j, 7.6743e-04-1.3451e-04j, 2.3050e-03-4.0369e-04j,
        4.0286e-03-7.0560e-04j, 1.2701e-03-2.2233e-04j, 7.0158e-04-1.2203e-04j,
        1.6565e-03-2.8970e-04j, 3.1274e-03-5.5190e-04j, 1.6939e-03-2.9270e-04j,
        3.9962e-03-6.9318e-04j, 1.4591e-02-2.5479e-03j, 1.0131e-02-1.7808e-03j,
        2.6264e-03-4.5244e-04j, 1.0535e-03-1.8284e-04j, 1.7321e-03-3.0183e-04j,
        9.5689e-04-1.6853e-04j, 3.5649e-

In [68]:
energy, psi_true = H.calc_ground(param=1)
psi_true = torch.tensor(psi_true)
print(psi_true)

tensor([ 0.4591, -0.1292, -0.1292,  0.0774, -0.1292,  0.0404,  0.0774, -0.0607,
        -0.1292,  0.0378,  0.0404, -0.0274,  0.0774, -0.0274, -0.0607,  0.0564,
        -0.1292,  0.0375,  0.0378, -0.0246,  0.0404, -0.0137, -0.0274,  0.0247,
         0.0774, -0.0246, -0.0274,  0.0221, -0.0607,  0.0247,  0.0564, -0.0607,
        -0.1292,  0.0378,  0.0375, -0.0246,  0.0378, -0.0128, -0.0246,  0.0221,
         0.0404, -0.0128, -0.0137,  0.0110, -0.0274,  0.0113,  0.0247, -0.0274,
         0.0774, -0.0246, -0.0246,  0.0192, -0.0274,  0.0110,  0.0221, -0.0246,
        -0.0607,  0.0221,  0.0247, -0.0246,  0.0564, -0.0274, -0.0607,  0.0774,
        -0.1292,  0.0404,  0.0378, -0.0274,  0.0375, -0.0137, -0.0246,  0.0247,
         0.0378, -0.0128, -0.0128,  0.0113, -0.0246,  0.0110,  0.0221, -0.0274,
         0.0404, -0.0137, -0.0128,  0.0110, -0.0137,  0.0060,  0.0110, -0.0137,
        -0.0274,  0.0110,  0.0113, -0.0128,  0.0247, -0.0137, -0.0274,  0.0404,
         0.0774, -0.0274, -0.0246,  0.02

In [69]:
mse = torch.mean((psi_predicted - psi_true) ** 2)
print("MSE:", mse)

MSE: tensor(0.0034+6.9522e-05j, dtype=torch.complex128, grad_fn=<MeanBackward0>)


In [70]:
real_imag = torch.view_as_real(psi_predicted)
psi_true_real_imag = torch.view_as_real(psi_true.to(torch.complex64))
print(real_imag)
print(psi_true_real_imag)

tensor([[ 1.1880e-01, -2.1194e-02],
        [ 2.9666e-02, -5.2249e-03],
        [ 1.1689e-02, -2.0531e-03],
        [ 1.8243e-02, -3.2033e-03],
        [ 1.0131e-02, -1.7808e-03],
        [ 3.5269e-03, -6.0937e-04],
        [ 5.5343e-03, -9.6709e-04],
        [ 1.4591e-02, -2.5479e-03],
        [ 9.7589e-03, -1.7160e-03],
        [ 2.7122e-03, -4.7215e-04],
        [ 1.2949e-03, -2.2536e-04],
        [ 2.5236e-03, -4.3972e-04],
        [ 4.2755e-03, -7.4673e-04],
        [ 1.9544e-03, -3.3704e-04],
        [ 4.1603e-03, -7.2152e-04],
        [ 1.3630e-02, -2.3707e-03],
        [ 9.7589e-03, -1.7160e-03],
        [ 2.5721e-03, -4.4465e-04],
        [ 1.0673e-03, -1.8645e-04],
        [ 1.8286e-03, -3.2076e-04],
        [ 1.0926e-03, -1.9202e-04],
        [ 4.3595e-04, -7.5161e-05],
        [ 7.6743e-04, -1.3451e-04],
        [ 2.3050e-03, -4.0369e-04],
        [ 4.0286e-03, -7.0560e-04],
        [ 1.2701e-03, -2.2233e-04],
        [ 7.0158e-04, -1.2203e-04],
        [ 1.6565e-03, -2.897

In [71]:
mse = torch.mean((real_imag - psi_true_real_imag) ** 2)
print("MSE:", mse)

MSE: tensor(0.0017, grad_fn=<MeanBackward0>)


In [72]:
adam = torch.optim.Adam(model.parameters(), lr=0.01)
adam.zero_grad()
mse.backward(retain_graph=True)
adam.step()

Being able to produce a computational graph provides evidence that .backward actually does full backpropagation:


In [73]:
from torchviz import make_dot

In [74]:
# By user ucalyptus, from https://github.com/szagoruyko/pytorchviz/issues/41
def resize_graph(dot, size_per_element=0.15, min_size=12):
    """Resize the graph according to how much content it contains.
    Modify the graph in place.
    """
    # Get the approximate number of nodes and edges
    num_rows = len(dot.body)
    content_size = num_rows * size_per_element
    size = max(min_size, content_size)
    size_str = str(size) + "," + str(size)
    dot.graph_attr.update(size=size_str)

In [75]:
model.named_parameters()

<generator object Module.named_parameters at 0x7cf50360b940>

In [76]:
graph = make_dot(
    mse, params=dict(model.named_parameters()), show_attrs=True, show_saved=True
)
resize_graph(graph, 0.7)
graph.render("mse_full", format="png")

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [32, 2]], which is output 0 of AsStridedBackward0, is at version 4; expected version 3 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).